NCF Recommender with Explict Feedback

In [1]:
#Context-Aware Neural Collaborative Filtering (CA-NCF)
#Author: Isam Mashhour Al Jawarneh
#Date: 30 January 2019

from zoo.models.recommendation import *
from zoo.models.recommendation.utils import *
from zoo.common.nncontext import init_nncontext
import os
import sys
import datetime as dt
from bigdl.dataset.transformer import *
from bigdl.dataset.base import *
from bigdl.nn.criterion import *
from bigdl.optim.optimizer import *
from bigdl.optim import *
from bigdl.util.common import *
import matplotlib

matplotlib.use('agg')
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
sc = init_nncontext(create_spark_conf().setAppName("CA_NCF Tripadvisor"))

In [3]:
#necessary libraries
from scipy import stats
import numpy as np
import pandas as pd

In [4]:
#using pandas to read data from csv file
df = pd.read_csv("datasets/DePaulMovie/ratings.csv", names=['UserID', 'ItemID', 'rating','Time','Location','Companion'], header=None, usecols=[0,1,2,3,4,5])[['UserID', 'ItemID', 'rating','Time','Location','Companion']]

In [5]:
# for constructing objects containing items, the contexts and context conditions
class MyClass(object):
    def __init__(self, item,condition,context):
        self.number = number
        self.item = item
        self.condition = condition
        self.context = context

In [6]:
######ITEM SPLITTING Procedure############
## as described in :
## Experimental evaluation of context-dependent collaborative filtering using item splitting
## https://dl.acm.org/citation.cfm?id=2579997
#df = df1.head(1000)
## output a list (item, condition, context), where condition is the context condition that will be used 
##for splitting the item

#profiling data
some_list = []
contextFactors = ['Time','Location','Companion']
dflist = df['ItemID'].unique().tolist()

cmax = 0.0
con = ''
context = ''
for i in dflist:
    cmax = 0.0
    for j in contextFactors:
        #print(j)
        conditionList = df[j].unique().tolist()
        #array_length1 = len(conditionList)
        #print(array_length1)
        for k in conditionList:
            #print(k)
            item1 = df[(df['ItemID'] == i) & (df[j] == k)]
            item1.reset_index(inplace= True)  
            item2 = df[(df['ItemID'] == i) & (df[j] != k)]
            item2.reset_index(inplace= True)
            l = stats.ttest_ind(item1['rating'], item2['rating'])
            #if l.statistic > 4.0:
            #print(i), print(l)
            if (l.statistic > cmax) :
                cmax = l.statistic
                con = k
                context = j
            #print(cmax)
        
    #print('finished')
    if cmax > 2.5:
        print('item ' + str(i) + ' cmax is ' + str(cmax) + ' con ' + str(con) + ' context ' + str(context))
        some_list.append(MyClass(str(i),str(con),str(context)))

      
  



/home/isam/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:3506: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/home/isam/.local/lib/python3.5/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


item tt2251217 cmax is 3.412806954531787 con Home context Location
item tt3203616 cmax is 2.9541957835039856 con Weekday context Time


In [7]:
import os
#should be the same path in Jupyter where this notebook exists
notebook_path = os.path.abspath("CA_NCF_Movie_DePaulMovie.ipynb")

In [8]:
sqlContext = SQLContext(sc)
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import col, udf

#loading CSV data into a DataFrame

Rating = Row("UserID", "ItemID", "Rating","Time","Location","Companion")

ratingsget = os.path.join(os.path.dirname(notebook_path), "datasets/DePaulMovie/ratings.csv")
ratings= sc.textFile(ratingsget) \
    .map(lambda line: line.split(",")[0:6])\
    .map(lambda line: (line[0], line[1], int(line[2]), line[3],line[4],line[5]))\
    .map(lambda r: Rating(*r))
ratingDF_OLD_VERY = sqlContext.createDataFrame(ratings)
ratingDF_OLD_VERY.show(5)

someDFtt = ratingDF_OLD_VERY.select(col("UserID")).distinct()

print("UserID :   " + str(someDFtt.count()))

someDFtt1 = ratingDF_OLD_VERY.select(col("ItemID")).distinct()

print("ItemID :   " + str(someDFtt1.count()))


+------+---------+------+----+--------+---------+
|UserID|   ItemID|Rating|Time|Location|Companion|
+------+---------+------+----+--------+---------+
|  1123|tt1499658|     2|  NA|      NA|       NA|
|  1123|tt0405422|     4|  NA|      NA|       NA|
|  1123|tt0109830|     5|  NA|      NA|       NA|
|  1123|tt0088763|     3|  NA|      NA|       NA|
|  1123|tt0133093|     3|  NA|      NA|       NA|
+------+---------+------+----+--------+---------+
only showing top 5 rows

UserID :   97
ItemID :   79


In [9]:
####all the following cells are preparations for incorporating context into original items
####we are actual constructing first new artificial items set
####then we apply the NCF form the library as-is

In [10]:
populateList = []
def condition(i):
    switcher={
            'Home':["Home", "Cinema","NA"],
            'Cinema':["Cinema", "Home","NA"],
            'Weekday':["Weekday", "Weekend", "NA"],
            'Weekend':["Weekend", "Weekday", "NA"],
            'weekend':["weekend", "weekday"],
            'Partner':["Partner", "Family", "Alone","NA"],
            'Family':["Family", "Partner", "Alone","NA"],
            'Alone':["Alone", "Partner", "Family","NA"],
           
         }
    return switcher.get(i,"Invalid year")
    

In [11]:
from pyspark.sql.functions import col, udf
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, Binarizer
from pyspark.sql.functions import rand 

#transformations

indexer = StringIndexer(inputCol="UserID", outputCol="newUserID1")
indexer1 = StringIndexer(inputCol="ItemID", outputCol="indexedItemID2")
indexer2 = StringIndexer(inputCol="artItemID1", outputCol="artItemID2")
binarizer22 = Binarizer(threshold=2.0, inputCol="locationIndexed", outputCol="binarized_location")

In [12]:
pipeline = Pipeline(stages=[indexer1])
                
ratingDF_OLD = pipeline.fit(ratingDF_OLD_VERY).transform(ratingDF_OLD_VERY)

In [13]:
#ratingDF_OLD.show(2)

In [14]:
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, udf
df_list = []
toDiscard_df_list = []
#constructing the dataFrame
for obj in some_list:
    itemid= obj.item
    ctxCond = obj.condition
    ctx = obj.context
    partialDF = ratingDF_OLD.withColumn("newUserID1",col("UserID")).select("*").where((col("ItemID")==itemid) )
    toDiscard_df_list.append(partialDF)
    #print(partialDF.count())
    populateList = condition(ctxCond)
    print(populateList)
    print(ctx)
    general_udf = udf(lambda somename: categorical_from_vocab_list(somename, populateList, start=1))
    partialDF_indexed = partialDF.withColumn("locationIndexed", general_udf(col(ctx)).cast("double"))
    #print(partialDF_indexed.where(col('locationIndexed') != 0.0).count())
    #print(partialDF_indexed.count())
    pipeline1 = Pipeline(stages = [binarizer22])
    allDF_temp =  pipeline1.fit(partialDF_indexed).transform(partialDF_indexed)
    allDF_temp.select('ItemID','binarized_location','Time','Location','Companion','indexedItemID2').show(5)

    bucket_cross_udf = udf(lambda feature1, feature2: hash_bucket(str(feature1) + "_" + str(feature2), bucket_size=10000))
    df_list.append(allDF_temp.withColumn("artItemID1", bucket_cross_udf(col("indexedItemID2"), col("binarized_location")).cast("int") + 1))

['Home', 'Cinema', 'NA']
Location
+---------+------------------+-------+--------+---------+--------------+
|   ItemID|binarized_location|   Time|Location|Companion|indexedItemID2|
+---------+------------------+-------+--------+---------+--------------+
|tt2251217|               1.0|     NA|      NA|       NA|          67.0|
|tt2251217|               1.0|     NA|      NA|       NA|          67.0|
|tt2251217|               0.0|Weekday|  Cinema|    Alone|          67.0|
|tt2251217|               0.0|Weekday|  Cinema|   Family|          67.0|
|tt2251217|               0.0|Weekday|  Cinema|  Partner|          67.0|
+---------+------------------+-------+--------+---------+--------------+
only showing top 5 rows

['Weekday', 'Weekend', 'NA']
Time
+---------+------------------+-------+--------+---------+--------------+
|   ItemID|binarized_location|   Time|Location|Companion|indexedItemID2|
+---------+------------------+-------+--------+---------+--------------+
|tt3203616|               1.0| 

In [15]:
allDF_prepare = reduce(DataFrame.unionAll, df_list).cache()


In [16]:
#allDF_prepare.show(1)

In [17]:
#toDiscard_df_all_sub.show(2)

In [18]:
from pyspark.sql.functions import lit
toDiscard_df_all = reduce(DataFrame.unionAll, toDiscard_df_list).cache()
#print(toDiscard_df_all.select(col('artItemID1')).distinct().count())
toDiscard_df_all_sub = toDiscard_df_all.select("*")
ratingDF_OLD_sub = ratingDF_OLD.withColumn("newUserID1",col("UserID"))
remainingDF = ratingDF_OLD_sub.subtract(toDiscard_df_all_sub)

remainingDF_indexed = remainingDF.withColumn('binarized_location',lit(1.0))
bucket_cross_udf1 = udf(lambda feature1, feature2: hash_bucket(str(feature1) + "_" + str(feature2), bucket_size=10000))
remainingDF_indexed_all = remainingDF_indexed.withColumn("artItemID1", bucket_cross_udf1(col("ItemID"), col("binarized_location")).cast("int") + 1)

In [19]:
toDiscard_df_all_sub.show(2)

+------+---------+------+----+--------+---------+--------------+----------+
|UserID|   ItemID|Rating|Time|Location|Companion|indexedItemID2|newUserID1|
+------+---------+------+----+--------+---------+--------------+----------+
|  1020|tt2251217|     2|  NA|      NA|       NA|          67.0|      1020|
|  1074|tt2251217|     4|  NA|      NA|       NA|          67.0|      1074|
+------+---------+------+----+--------+---------+--------------+----------+
only showing top 2 rows



In [20]:
ratingDF_OLD_sub.show(2)

+------+---------+------+----+--------+---------+--------------+----------+
|UserID|   ItemID|Rating|Time|Location|Companion|indexedItemID2|newUserID1|
+------+---------+------+----+--------+---------+--------------+----------+
|  1123|tt1499658|     2|  NA|      NA|       NA|          38.0|      1123|
|  1123|tt0405422|     4|  NA|      NA|       NA|          18.0|      1123|
+------+---------+------+----+--------+---------+--------------+----------+
only showing top 2 rows



In [21]:

remainingDF_indexed_all_sub = remainingDF_indexed_all.select(col('newUserID1') ,col('indexedItemID2'),col("UserID"),
                                        col('Rating'),col('ItemID'),col('artItemID1'),col("binarized_location"))
allDF_prepare_sub_sub = allDF_prepare.select(col("newUserID1") ,col("indexedItemID2"),col("UserID"),
                                        col("Rating"),col("ItemID"),col("artItemID1"),col("binarized_location"))
allDF_difinite = remainingDF_indexed_all_sub.union(allDF_prepare_sub_sub).cache()

In [22]:
someDF_art = allDF_difinite.select(col("artItemID1")).distinct()
print(someDF_art.count())

81


In [23]:
pipeline2 = Pipeline(stages = [indexer2])

allDF =  pipeline2.fit(allDF_difinite).transform(allDF_difinite)

someDF1 = allDF.select((col("newUserID1").cast("int") ).alias("newUserID"),col("indexedItemID2").cast("int"),
                       col("Rating"),col("ItemID"),(col("artItemID2").cast("int") + 1).alias("aritid"))
someDF2 = allDF.select((col("newUserID1").cast("int")).alias("newUserID"),
                       (col("artItemID2").cast("int") + 1).alias("artItemID"),col("Rating"))#.distinct()

someDF_OLD = someDF2.orderBy(rand())#.select("*").limit(5000000)


ratingDF_indexed1 = allDF.select((col("newUserID1").cast("int")).alias("usrID"),col("UserID"),
                                 col("indexedItemID2").cast("int"),col("Rating"),col("ItemID"),
                                 col("binarized_location"), 
                                 (col("artItemID2") .cast("int") + 1).alias("aritid"))

ratingDF_indexed2 = allDF.select((col("newUserID1").cast("int")).alias("usrID"),
                                 (col("indexedItemID2").cast("int") + 1).alias("indexedItemID1"),
                                 col("Rating"))#.distinct()

CARSkit_DF = ratingDF_indexed1.select(col("usrID").alias("userID"),col("ItemID"),  col("Rating"),
                                      col("binarized_location"))
CARSkit_DF_path = os.path.join(os.path.dirname(notebook_path), "datasets/tripAdvisor/")
CARSkit='CARSkit-'+dt.datetime.now().strftime("%Y%m%d-%H%M%S")
CARSkit_DF.coalesce(1).write.csv(CARSkit_DF_path + CARSkit + '.csv')

#shuffling data
ratingDF_indexed_OLD = ratingDF_indexed2.orderBy(rand())


In [24]:
# discard items with less than 20 ratings (interactions)
import pyspark.sql.functions as f
from pyspark.sql import Window
w = Window.partitionBy('artItemID')
someDF_NEW = someDF_OLD.withColumn('count_artItemID', f.count('newUserID').over(w))\
    .where(f.col('count_artItemID') > 5)\
    .drop('count_artItemID')#.select("*").limit(5000000)


In [25]:
w = Window.partitionBy('newUserID')
someDF = someDF_NEW.withColumn('count_newUserID', f.count('artItemID').over(w))\
.where(f.col('count_newUserID') > 5).drop('count_newUserID')

In [26]:
#someDF.count()

In [27]:
#TWEAKABLE: drop more items with less than a threshold to account for sparsity
import pyspark.sql.functions as f
from pyspark.sql import Window
w = Window.partitionBy('indexedItemID1')
ratingDF_indexed = ratingDF_indexed_OLD.withColumn('count_ItemID', f.count('usrID').over(w))\
    .where(f.col('count_ItemID') > 5)\
    .drop('count_artItemID')

In [28]:
#ratingDF_indexed.count()

In [29]:
someDFt3 = someDF.select(col("newUserID")).distinct()
max_user_id1 =  someDF.agg({"newUserID": "max"}).collect()[0][0]
someDFt4 = someDF.select(col("artItemID")).distinct()
max_item_id1 = someDF.agg({"artItemID": "max"}).collect()[0][0]
print(max_user_id1, max_item_id1)

1122 80


comprehensing your data. 
a tuple is (userid, movieid, rating). 

In [30]:
someDFt = ratingDF_indexed.select(col("usrID")).distinct()

print("usrID :   " + str(someDFt.count()))

max_user_id = ratingDF_indexed.agg({"usrID": "max"}).collect()[0][0]

someDFt1 = ratingDF_indexed.select(col("indexedItemID1")).distinct()

max_item_id  = ratingDF_indexed.agg({"indexedItemID1": "max"}).collect()[0][0]
print(max_user_id, max_item_id)

usrID :   97
1123 79


In [31]:
####now that we have constructed artificial items set, we are then ready to apply NCF from BigDL

Transforming raw rarings into an RDD . 
BigDl 6.0 requires input as RDD of samples, BigDL data structure that can be formed by two numpy arrays
containing features and lables for the first and second array, respectively.
in the following format Sample.from_ndarray(feature, label)


In [32]:

def build_sample(userid, itemid, rating):
    sampleTaken = Sample.from_ndarray(np.array([userid, itemid]), np.array([rating]))
    return UserItemFeature(userid, itemid, sampleTaken)
#CA-NCF settings
pairFeatureRdds_CANCF = someDF.rdd.map(lambda x: build_sample(x[0], x[1],x[2]))
pairFeatureRdds_CANCF.repartition(4)


pairFeatureRdds_CANCF.take(3)



In [33]:
#80% training data, 20% testing, randomly split

import time
millis = int(round(time.time() * 1000))

#CANCF settings
#tweakable, seed= 42L
trainPairFeatureRdds_CANCF, valPairFeatureRdds_CANCF = pairFeatureRdds_CANCF.randomSplit([0.8, 0.2], seed = millis)
valPairFeatureRdds_CANCF.cache()
train_rdd_CANCF= trainPairFeatureRdds_CANCF.map(lambda pair_feature: pair_feature.sample)
val_rdd_CANCF= valPairFeatureRdds_CANCF.map(lambda pair_feature: pair_feature.sample)


applying Nerual Collaborative filtering (He, 2015)

In [34]:
#Neural Collaborative Filtering (NCF) model in BigDL 6.0
#see more parameters here:
#https://github.com/intel-analytics/analytics-zoo/blob/master/docs/docs/APIGuide/Models/recommendation.md

#more hyperparameters  ,user_embed=40, item_embed=2
ncf = NeuralCF(user_count=max_user_id1, item_count=max_item_id1, class_num=5, hidden_layers=[20,10])#, include_mf = True,mf_embed = 10)

creating: createZooNeuralCF


In [35]:
##predicting user-item pairs using CA-NCF
predictUserItem_CANCF = ncf.predict_user_item_pair(valPairFeatureRdds_CANCF)
for ans in predictUserItem_CANCF.take(5): print(ans)

UserItemPrediction [user_id: 1025, item_id: 44, prediction: 4, probability: 0.23324161922705944]
UserItemPrediction [user_id: 1025, item_id: 10, prediction: 4, probability: 0.23419799421693552]
UserItemPrediction [user_id: 1025, item_id: 14, prediction: 4, probability: 0.23587748289700058]
UserItemPrediction [user_id: 1084, item_id: 1, prediction: 4, probability: 0.2362773049811907]
UserItemPrediction [user_id: 1084, item_id: 1, prediction: 4, probability: 0.2362773049811907]


In [36]:
#Recommending 5 items for every user
recItemsForUsers_CANCF = ncf.recommend_for_user(valPairFeatureRdds_CANCF, 5)
for ans in recItemsForUsers_CANCF.take(10): print(ans)

UserItemPrediction [user_id: 1001, item_id: 19, prediction: 4, probability: 0.23597276858863753]
UserItemPrediction [user_id: 1001, item_id: 19, prediction: 4, probability: 0.23597276858863753]
UserItemPrediction [user_id: 1001, item_id: 11, prediction: 4, probability: 0.2355444824288736]
UserItemPrediction [user_id: 1001, item_id: 7, prediction: 4, probability: 0.23535668014321895]
UserItemPrediction [user_id: 1001, item_id: 5, prediction: 4, probability: 0.2352673925995903]
UserItemPrediction [user_id: 1002, item_id: 7, prediction: 4, probability: 0.23542826367545852]
UserItemPrediction [user_id: 1002, item_id: 66, prediction: 4, probability: 0.2348852112055863]
UserItemPrediction [user_id: 1002, item_id: 43, prediction: 4, probability: 0.23479739006154166]
UserItemPrediction [user_id: 1002, item_id: 43, prediction: 4, probability: 0.23479739006154166]
UserItemPrediction [user_id: 1002, item_id: 1, prediction: 4, probability: 0.23424573984901648]
